In [7]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import XMLOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.chat_models import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()


os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] =os.getenv("GROQ_API_KEY")

##Langsmith Tracking and tracing
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] =os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [6]:
# 1. Define the output schema using Pydantic
class Person(BaseModel):
    name: int = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")

# 2. Create an XML output parser for the schema
parser = XMLOutputParser(pydantic_object=Person)

# 3. Define the prompt with parser format instructions
prompt = PromptTemplate(
    template=(
        "Extract the person's name and age from the following sentence:\n\n{text}\n\n"
        "Return the result in the following XML format:\n\n{format_instructions}"
    ),
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# 4. Set up the LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)

# 5. Pipe the components: prompt → llm → parser
chain = prompt | llm | parser

# 6. Run the chain with input text
output = chain.invoke({"text": "My name is Bob and I am 45 years old."})

print(output)

{'person': [{'name': 'Bob'}, {'age': '45'}]}


In [32]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser

class Product(BaseModel):
    name: str = Field(..., description="The product's name")
    details: str = Field(..., description="The product's details")
    tentativePrice: float = Field(..., description="The product's tentative price")


# 2. Create a JSON output parser for the schema
parser = JsonOutputParser(pydantic_object=Product)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that extracts product information and formats it as JSON."),
    ("user", "Extract the product name, details and tentative price from this text and return it as a valid JSON object with 'name', 'details', and 'tentativePrice' fields:\n\n{input}\n\n{format_instructions}")
])

# 4. Set up the LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)

# 5. Pipe the components: prompt → llm → parser
chain = prompt | llm | parser

# 6. Run the chain with input text
output = chain.invoke({
    "input": "I want to buy a new phone. I am looking for a phone that has a good camera. I am willing to pay $500 for it.",
    "format_instructions": parser.get_format_instructions()
})

print(output)

{'name': 'phone', 'details': 'a phone that has a good camera', 'tentativePrice': 500}
